In [ ]:
!pip install Ninja

In [ ]:
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing.git

In [ ]:
!git clone https://github.com/Healthcare-Robotics/bodies-at-rest.git

In [ ]:
%cd Self-Correction-Human-Parsing

In [5]:
!mkdir input
!mkdir output

In [ ]:
%cd ..
%cd /content/bodies-at-rest/PressurePose

In [ ]:
!chmod +x download_real.sh
!bash download_real.sh

In [22]:
import pandas as pd

object = pd.read_pickle(r'/content/bodies-at-rest/data_BR/real/S103/prescribed.p')

In [28]:
from PIL import Image
import numpy as np

w, h = 880, 440
data = np.zeros((h, w, 3), dtype=np.uint8)
data = object['RGB'][0]
img = Image.fromarray(data, 'RGB')
img.save('my.png')
img.show()

In [27]:
object['RGB'][0].shape

(880, 440, 3)

In [30]:
!python '/content/Self-Correction-Human-Parsing/simple_extractor.py' --dataset 'pascal' --model-restore '/content/drive/MyDrive/models/exp-schp-201908270938-pascal-person-part.pth' --input-dir '/content/Self-Correction-Human-Parsing/input/' --output-dir '/content/Self-Correction-Human-Parsing/output'

Evaluating total class number 7 with ['Background', 'Head', 'Torso', 'Upper Arms', 'Lower Arms', 'Upper Legs', 'Lower Legs']
100% 1/1 [00:00<00:00,  2.17it/s]
